In [46]:
import requests, re
from datetime import datetime
from queue import Queue
from bs4 import BeautifulSoup as bs
from threading import Thread
from time import  sleep

In [42]:
def parseCookieRaw(cookies: str) -> dict[str]:
    result = dict()
    cookiesArr = cookies.split(";")
    cookiesArr = [cookieItem.strip() for cookieItem in cookiesArr]
    for cookieItem in cookiesArr:
        cookieItemArr = cookieItem.split("=")
        result[cookieItemArr[0]] = cookieItemArr[1]
    return result

In [43]:
def convertDateToTimestamp(input_year = datetime.now().year):
    if input_year == datetime.now().year:
        day = datetime.now().day
        month = datetime.now().month
        date_string = f'{month}/{day}/{input_year}, 11:59:59 PM'
    else:
        date_string = f'12/31/{input_year}, 11:59:59 PM'
    date_format = '%m/%d/%Y, %I:%M:%S %p'
    timestamp = datetime.strptime(date_string, date_format).replace(microsecond=0).timestamp()
    return str(int(timestamp))

In [45]:
def getFb_dtsg(cookies: dict[str]) -> str:
    HEADERS_GET_FB_DTSG = {
        'authority': 'mbasic.facebook.com',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'vi-VN,vi;q=0.9,fr-FR;q=0.8,fr;q=0.7,en-US;q=0.6,en;q=0.5',
        'cache-control': 'no-cache',
        'pragma': 'no-cache',
        'sec-ch-ua': '"Chromium";v="112", "Google Chrome";v="112", "Not:A-Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'none',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
      }
    try:
      response = requests.get('https://mbasic.facebook.com/me/', cookies=cookies, headers=HEADERS_GET_FB_DTSG)
      soup = bs(response.content, 'html.parser')
      fb_dtsg = soup.find('input', {'name': 'fb_dtsg'}).get('value')
      return fb_dtsg
    except Exception as e:
      print(e)
      return


In [63]:
def getListPostOnFacebook(cookies: dict[str], fb_dtsg: str, year: int, list_Post_Queue: Queue):
  uid = cookies['c_user']
  headers = {
      'authority': 'www.facebook.com',
      'accept': '*/*',
      'accept-language': 'en-US,en;q=0.9,vi;q=0.8',
      'content-type': 'application/x-www-form-urlencoded',
      'origin': 'https://www.facebook.com',
      'referer': 'https://www.facebook.com/truongnhudatt/',
      'sec-ch-prefers-color-scheme': 'light',
      'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
      'sec-ch-ua-full-version-list': '"Not.A/Brand";v="8.0.0.0", "Chromium";v="114.0.5735.134", "Google Chrome";v="114.0.5735.134"',
      'sec-ch-ua-mobile': '?0',
      'sec-ch-ua-platform': '"Windows"',
      'sec-ch-ua-platform-version': '"10.0.0"',
      'sec-fetch-dest': 'empty',
      'sec-fetch-mode': 'cors',
      'sec-fetch-site': 'same-origin',
      'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
      'viewport-width': '935',
      'x-asbd-id': '129477',
      'x-fb-friendly-name': 'ProfileCometTimelineFeedRefetchQuery',
      'x-fb-lsd': 'FPaahspEeikWFdUA1nBFxJ',
  }
  end_cursor = ""
  has_next_page = True
  beforeTime = convertDateToTimestamp(year)
  try:
    while has_next_page:
      data = {
          'av': uid ,
          'fb_dtsg': fb_dtsg,
          'jazoest': '25633',
          'lsd': 'FPaahspEeikWFdUA1nBFxJ',
          'fb_api_caller_class': 'RelayModern',
          'fb_api_req_friendly_name': 'ProfileCometTimelineFeedRefetchQuery',
          'variables': '{"UFI2CommentsProvider_commentsKey":"ProfileCometTimelineRoute","afterTime":null,"beforeTime":'+beforeTime+',"count":3,"cursor":"'+end_cursor+'","displayCommentsContextEnableComment":null,"displayCommentsContextIsAdPreview":null,"displayCommentsContextIsAggregatedShare":null,"displayCommentsContextIsStorySet":null,"displayCommentsFeedbackContext":null,"feedLocation":"TIMELINE","feedbackSource":0,"focusCommentID":null,"memorializedSplitTimeFilter":null,"omitPinnedPost":true,"postedBy":{"group":"OWNER"},"privacy":{"exclusivity":"INCLUSIVE","filter":"ALL"},"privacySelectorRenderLocation":"COMET_STREAM","renderLocation":"timeline","scale":1,"stream_count":10,"taggedInOnly":null,"useDefaultActor":false,"id":"'+uid+'","__relay_internal__pv__GroupsCometDelayCheckBlockedUsersrelayprovider":false,"__relay_internal__pv__IsWorkUserrelayprovider":false,"__relay_internal__pv__IsMergQAPollsrelayprovider":false,"__relay_internal__pv__StoriesArmadilloReplyEnabledrelayprovider":false,"__relay_internal__pv__StoriesRingrelayprovider":false}',
          'server_timestamps': 'true',
          'doc_id': '9321508997924101',
      }
      response = requests.post('https://www.facebook.com/api/graphql/', cookies=cookies, headers=headers, data=data)
      storyID = re.findall(r'"id":"(U[^"]+=)","comet_sections', response.text)
      if storyID:
        print(storyID)
        # _ = list(map(list_Post_Queue.put, id_match))
        break
      else:
        break
      has_next_page_match = re.search(r'"has_next_page":(\w+)', response.text)
      if has_next_page_match:
        has_next_page = bool(has_next_page_match.group(1))
      end_cursor_match = re.search(r'"end_cursor":"([^"]+)"', response.text)
      if end_cursor_match:
        end_cursor = end_cursor_match.group(1)
  except Exception as e:
    print(e)

In [64]:
cookies = parseCookieRaw("sb=ttpvZI7d9xfsOc9zFyV0vW4l; dpr=1.100000023841858; usida=eyJ2ZXIiOjEsImlkIjoiQXJ3cDVudDFuOG15dDciLCJ0aW1lIjoxNjg3NTA4MjAxfQ%3D%3D; locale=vi_VN; datr=k56WZKQ1Pvs0DBT_sPzmFhb2; c_user=100013323961498; xs=47%3A9eTlA0cT9kzJPg%3A2%3A1687592672%3A-1%3A6253; fr=0UZz7aQXaYkhvU7Ez.AWVf7IjgAxEUWBTjDW5e5v99pB4.Bkln8R.a_.AAA.0.0.Bklp7i.AWV8oczjuP4; wd=1065x861; presence=C%7B%22t3%22%3A%5B%7B%22i%22%3A%22g.4088083211206594%22%7D%5D%2C%22utc3%22%3A1687593312166%2C%22v%22%3A1%7D")
fb_dtsg = getFb_dtsg(cookies)
list_Post_Queue = Queue()
getListPostOnFacebook(cookies, fb_dtsg, 2023, list_Post_Queue)

['UzpfSTEwMDAxMzMyMzk2MTQ5ODoxNzY3MDk5Nzk3MDc3NDY5OjE3NjcwOTk3OTcwNzc0Njk=', 'UzpfSTEwMDAxMzMyMzk2MTQ5ODoxNzQyNTMzNzI5NTM0MDc2OjE3NDI1MzM3Mjk1MzQwNzY=', 'UzpfSTEwMDAxMzMyMzk2MTQ5ODoxNzE3MzYxOTY4NzE3OTE5OjE3MTczNjE5Njg3MTc5MTk=']


In [ ]:
def removePost(cookies: dict[str],fb_dtsg: str,  list_Post_Queue: Queue):
    headers = {
        'authority': 'www.facebook.com',
        'accept': '*/*',
        'accept-language': 'en-US,en;q=0.9',
        'content-type': 'application/x-www-form-urlencoded',
        'origin': 'https://www.facebook.com',
        'referer': 'https://www.facebook.com',
        'sec-ch-prefers-color-scheme': 'light',
        'sec-ch-ua': '"Chromium";v="103", ".Not/A)Brand";v="99"',
        'sec-ch-ua-full-version-list': '"Chromium";v="103.0.5060.134", ".Not/A)Brand";v="99.0.0.0"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-ch-ua-platform-version': '"10.0.0"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.134 Safari/537.36',
        'viewport-width': '1132',
        'x-asbd-id': '129477',
        'x-fb-friendly-name': 'useCometTrashPostMutation',
        'x-fb-lsd': 'paVhHmefIHy5g8EsbMvA1c',
    }
    while True:
        idd = list_Post_Queue.get()
        if idd is None:
            continue
        elif idd == "Null":
            break
        else:
            story_id = getStoryIDFromUrl(cookies, idd)
            if story_id:
                data = {
                    'av': cookies['c_user'],
                    '__a': '1',
                    '__comet_req': '15',
                    'fb_dtsg': fb_dtsg,
                    'fb_api_caller_class': 'RelayModern',
                    'fb_api_req_friendly_name': 'useCometTrashPostMutation',
                    'variables': '{"input":{"story_id":"' + story_id + '","story_location":"TIMELINE","actor_id":"' +
                                 cookies['c_user'] + '","client_mutation_id":"1"}}',
                    'server_timestamps': 'true',
                    'doc_id': '7003056599769350',
                }
                try:
                    with requests.Session() as ses:
                        response = ses.post('https://www.facebook.com/api/graphql/', cookies=cookies, headers=headers,
                                            data=data)
                        print(response.text)
                except Exception as e:
                    print(e)
                    pass

In [35]:
list_Post_Queue = Queue()


[None, None, None]
